In [1]:
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras.layers import LSTM, Dropout
from keras.layers import TimeDistributed
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import sys
import heapq
import seaborn as sns
from pylab import rcParams
from keras import backend as K
print(K.tensorflow_backend._get_available_gpus())
%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 12, 5

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']


In [2]:
path = 'train_set.txt'
text = open(path).read()
print('corpus length:', len(text))
print(text[0:100])

text = text[:300000]

corpus length: 505330926
"Here be our forecast"
"Traveling to have a business meet take the fun out of the trip Especially if


In [3]:
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

print(f'unique chars: {len(chars)}')

unique chars: 55


In [4]:
SEQUENCE_LENGTH = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - SEQUENCE_LENGTH, step):
    sentences.append(text[i: i + SEQUENCE_LENGTH])
    next_chars.append(text[i + SEQUENCE_LENGTH])
print(f'num training examples: {len(sentences)}')

num training examples: 99987


In [ ]:
X = np.zeros((len(sentences), SEQUENCE_LENGTH, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [ ]:
sentences[100]

'e even desire or necessary As far as the'

In [ ]:
next_chars[100]

' '

In [ ]:
X[0][0]

array([False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [ ]:
y[0]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [ ]:
X.shape

(99987, 40, 55)

In [ ]:
y.shape

(99987, 55)

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(SEQUENCE_LENGTH, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               94208     
_________________________________________________________________
dense_1 (Dense)              (None, 55)                7095      
_________________________________________________________________
activation_1 (Activation)    (None, 55)                0         
Total params: 101,303
Trainable params: 101,303
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, y, validation_split=0.05, batch_size=128, epochs=20, shuffle=True).history

Instructions for updating:
Use tf.cast instead.
Train on 94987 samples, validate on 5000 samples
Epoch 1/20
94987/94987 [==============================] - 42s 447us/step - loss: 2.1318 - acc: 0.4066 - val_loss: 1.9247 - val_acc: 0.4484
Epoch 2/20
94987/94987 [==============================] - 41s 435us/step - loss: 1.6796 - acc: 0.5211 - val_loss: 1.7375 - val_acc: 0.5158
Epoch 3/20
94987/94987 [==============================] - 41s 433us/step - loss: 1.5166 - acc: 0.5668 - val_loss: 1.6385 - val_acc: 0.5430
Epoch 4/20
94987/94987 [==============================] - 42s 437us/step - loss: 1.4216 - acc: 0.5910 - val_loss: 1.6049 - val_acc: 0.5522
Epoch 5/20
94987/94987 [==============================] - 41s 436us/step - loss: 1.3592 - acc: 0.6072 - val_loss: 1.5860 - val_acc: 0.5528
Epoch 6/20
94987/94987 [==============================] - 41s 435us/step - loss: 1.3174 - acc: 0.6180 - val_loss: 1.5941 - val_acc: 0.5558
Epoch 7/20
94987/94987 [==============================] - 41s 433us/s

In [ ]:
model.save('keras_model.h5')
pickle.dump(history, open("history.p", "wb"))

In [ ]:
model = load_model('keras_model.h5')
history = pickle.load(open("history.p", "rb"))

In [ ]:
plt.plot(history['acc'])
plt.plot(history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left');

In [ ]:
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left');

In [ ]:
def prepare_input(text):
    x = np.zeros((1, SEQUENCE_LENGTH, len(chars)))
    for t, char in enumerate(text):
        x[0, t, char_indices[char]] = 1.
        return x

In [ ]:
prepare_input("This is an example of input for our LSTM".lower())

In [ ]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    
    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [ ]:
def predict_completion(text):
    original_text = text
    generated = text
    completion = ''
    while True:
        x = prepare_input(text)
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, top_n=1)[0]
        next_char = indices_char[next_index]
        text = text[1:] + next_char
        completion += next_char
        
        if len(original_text + completion) + 2 > len(original_text) and next_char == ' ':
            return completion

In [ ]:
def predict_completions(text, n=3):
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [indices_char[idx] + predict_completion(text[1:] + indices_char[idx]) for idx in next_indices]

In [ ]:
quotes = [
    "gas price change.I'm not upset that you lied to me, I'm upset that from now on I can't believe you",
    "That which does not kill us makes us stronger.",
    "I'm not upset that you lied to me, I'm upset that from now on I can't believe you.",
    "And those who were seen dancing were thought to be insane by those who could not hear the music.",
    "It is hard enough to remember my opinions, without also remembering my reasons for them!"
]

In [ ]:
for q in quotes:
    seq = q[:40].lower()
    print(seq)
    print(predict_completions(seq, 5))